In [ ]:
#| default_exp camera_talk
#| test: false

In [ ]:
#| export
#| eval: false

from __future__ import annotations

from collections import deque

import asyncio, datetime as dt, random, logging, os, contextlib


from agents import Agent, Runner
from agents.mcp import MCPServerSse         


log = logging.getLogger(__name__)

MCP_URL   = os.getenv("MCP_URL",   "http://tools:9001/sse")
CHART_MCP_URL  = os.getenv("CHART_MCP_URL", "http://chart:3000/sse")

LOCATION  = os.getenv("LOCATION", "Espoo,FI")   # city or "lat,lon"
HIST_LEN       = int(os.getenv("HIST_LEN", 20))

_series: deque[int] = deque(maxlen=HIST_LEN)
_labels: deque[str] = deque(maxlen=HIST_LEN)

last_val = 120

In [ ]:
#| export
#| eval: false

from agents import RunHooks
import html

    
class ToolChatHook(RunHooks[None]):
    def __init__(self, push_tool):
        self._push_tool = push_tool

    async def on_tool_start(self, ctx, agent, tool):
        self._push_tool(f"<div class='border-l-4 border-blue-500 pl-2'>"
                        f"🔧 {tool.name} started…</div>")

    async def on_tool_end(self, ctx, agent, tool, result):
        self._push_tool(f"<div class='border-l-4 border-green-500 pl-2'>"
                        f"✅ {tool.name} finished. ")
                        #f"{html.escape(str(result))}</div>")

    async def on_tool_error(self, ctx, agent, tool, err):
        self._push_tool(f"<div class='border-l-4 border-red-500 pl-2'>"
                        f"⚠️ {tool.name} failed: "
                        f"{html.escape(str(err))}</div>")

In [ ]:
#| export
#| eval: false

async def parking_camera_loop(q, sse_helper) -> None:
    """Send a Gen-UI block every 180 s with an updated chart."""
    global last_val                         # start full
    while True:
        delta      = random.randint(40, 60)  # random drop 30-50
        last_val   = max(last_val - delta, 0)
        stamp      = dt.datetime.now().strftime("%H:%M:%S")

        _series.append(last_val)
        _labels.append(stamp)

        try:
            ui_block = await asyncio.wait_for(
    _make_ui_block(
        list(_series), list(_labels), last_val, stamp,
        push_tool=lambda html: q.put_nowait(
            sse_helper("message",
                       f'<div hx-swap-oob="beforeend:#tool-log">{html}</div>')
        )
    ),
    timeout=180
)

        except Exception as exc:
                    log.error("parking widget failed: %s", exc, exc_info=True)
                    ui_block = (
                        f'<div hx-swap-oob="beforeend:#chatlog" '
                        f'class="alert alert-error">⚠ parking widget error – {exc}</div>'
                    )
        await q.put(sse_helper("message", ui_block))
        await asyncio.sleep(90)          # 3-min cadence

In [ ]:

#| export
#| eval: false

import json
from typing import Callable
from agents import ImageGenerationTool
from openai.types.responses.tool import ImageGeneration

async def _make_ui_block(series: list[int], labels: list[str], last_val, stamp, push_tool: Callable[[str], None]) -> str:
    """
    Generates a full GenUI HTML card via LLM using weather + parking.
    Ensures booking buttons and branding are always present.
    """
            # Prepare chart config dictionary and convert to JSON
    chart_config = {
            "chartConfig": {
                "type": "line",
                "data": {
                    "labels": labels,
                    "datasets": [{
                        "label": "Free slots",
                        "data": series,
                        "borderColor": "#10b981",
                        "fill": False  # Python bool – will be converted to JS-compatible false
                    }]
                },
                "options": {
                    "plugins": {
                        "title": {
                            "display": True,
                            "text": f"Parking last {len(series) * 3} min"
                        }
                    },
                    "scales": {
                        "y": {
                            "beginAtZero": True
                        }
                    }
                }
            },
            "width": 450,
            "height": 250,
            "devicePixelRatio": 2,
            "version": "4"
        }

        # Serialize to JSON with correct JS booleans
    chart_config_json = json.dumps(chart_config, indent=2)
    escaped_json = chart_config_json.replace("{", "{{").replace("}", "}}")
    async with MCPServerSse(name="ui", params={"url": MCP_URL}, client_session_timeout_seconds=300) as srv:

        instructions = f"""
        🎨 You are EcoGen, a generative-UI designer.

        You will craft one complete MonsterUI/Tailwind card and stream it via HTMX exactly once.

        Inputs this run
        • 🅿️  {last_val} free parking slots
        • 🕒  Current time: {stamp}
        • 🌤️  Live weather for “{LOCATION}”

        Step-by-step
        1. Call `weather.7_day_weather_forecast_for_coordinates` once.

        2. Call `chart.generate_chart` with the JSON below  
           (unchanged):

        ```json
        {escaped_json}
        ```
            

        3. Compose one HTML block wrapped in
        ```html
        <div hx-swap-oob="beforeend:#chatlog">
          …full card…
        </div>
        ```

        It must contain, in order:
        • A fun heading with an emoji  
        • Weather summary and your funny comment  
        • Parking status line, e.g. `🚗 {last_val} free slots`  
        • Chart    
        • A friendly encouragement paragraph to use bike or public transport  
        • Exactly three booking buttons, linking to:
          – CityBikes(HSL) → https://kaupunkipyorat.hsl.fi/en  
          – Donkey Republic → https://app.donkeyrepublic.com/#/reserve  
          – ListNRide → https://www.listnride.com/espoo  
          Use  
          `<div class="grid grid-cols-1 sm:grid-cols-3 gap-2 mt-4"> … </div>`

        • Branding footer:
        <div class="text-xs text-center text-gray-400 mt-3">
          Powered by EventTalks 🏒🌍
        </div>

        Rules:
        • HTML only – no Markdown, no JSON.  
        • Produce a single response; do not send drafts.  
        
        """
        
        
        
        agent = Agent(
            name="EcoGen Agent",
            model="o3",
            mcp_servers=[srv],
            #handoffs=[agent_image_generation],
            #    tools=[
        #ImageGenerationTool(
        #    tool_config={"type": "image_generation", "quality": "low"},
        #)
    #],
            instructions=instructions,  
            
            
        )

        hook = ToolChatHook(push_tool)      
        res  = await Runner.run(agent, input="", hooks=hook)
        
        log.info("Agent UI raw output: %s", res.final_output)

        ui = res.final_output.strip()
        
        

    # Safety net: wrap if LLM forgot
    if "hx-swap-oob" not in ui:
        ui = f'<div hx-swap-oob="beforeend:#chatlog">{ui}</div>'

    # Safety net: inject buttons if missing
    if all(x not in ui for x in ["City Bikes", "Donkey Republic", "ListNRide"]):
        ui += """
        <div class="grid grid-cols-1 sm:grid-cols-3 gap-2 mt-4">
          <a href="https://kaupunkipyorat.hsl.fi/en" class="btn btn-success btn-sm">🚲 City Bikes (HSL)</a>
          <a href="https://app.donkeyrepublic.com/#/reserve" class="btn btn-success btn-sm">🦄 Donkey Republic</a>
          <a href="https://www.listnride.com/espoo" class="btn btn-success btn-sm">🚴 ListNRide</a>
        </div>
        """

    # Branding if missing
    if "EventTalks" not in ui:
        ui += '<div class="text-xs text-center text-gray-400 mt-3">Powered by EventTalks 🏒🌍</div>'
    
    
    # Fallback: if no QuickChart chart was rendered, show local happy_bike.png
    # (QuickChart embeds an <img src="https://quickchart.io"…>, so we check for that)
    #if "quickchart.io" not in ui:
    #   ui += """
    #   <div class="mt-4 text-center">
    #     <img src="\\\\wsl.localhost\\Ubuntu\\home\\nata\\DataTalks\\happy_bike.png"
    #          alt="Happy bike"
    #          style="max-width:100%;height:auto;">
    #   </div>
    #   """
    
    
    log.info("parking widget: %s", ui)
    return ui




